<a href="https://colab.research.google.com/github/auberr/sparta_hp_ai/blob/main/sparta_AI_3_DistilBERT_prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DistilBERT fine-tuning으로 감정 분석 모델 학습하기

이번 실습에서는 pre-trained된 DistilBERT를 불러와 이전 주차 실습에서 사용하던 감정 분석 문제에 적용합니다. 먼저 필요한 library들을 불러옵니다.

In [6]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

그 후, 우리가 사용하는 DistilBERT pre-training 때 사용한 tokenizer를 불러옵니다.

In [7]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

DistilBERT의 tokenizer를 불러왔으면 이제 `collate_fn`과 data loader를 정의합니다. 이 과정은 이전 실습과 동일하게 다음과 같이 구현할 수 있습니다.

In [8]:
ds = load_dataset("stanfordnlp/imdb")


def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

이제 pre-trained DistilBERT를 불러옵니다. 이번에는 PyTorch hub에서 제공하는 DistilBERT를 불러봅시다.

In [9]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
model

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

출력 결과를 통해 우리는 DistilBERT의 architecture는 일반적인 Transformer와 동일한 것을 알 수 있습니다.
Embedding layer로 시작해서 여러 layer의 Attention, FFN를 거칩니다.

이제 DistilBERT를 거치고 난 `[CLS]` token의 representation을 가지고 text 분류를 하는 모델을 구현합시다.

In [10]:
from torch import nn


class TextClassifier(nn.Module):
  def __init__(self):
    super().__init__()

    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.classifier = nn.Linear(768, 1)

  def forward(self, x):
    x = self.encoder(x)['last_hidden_state']
    x = self.classifier(x[:, 0])

    return x


model = TextClassifier()

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


위와 같이 `TextClassifier`의 `encoder`를 불러온 DistilBERT, 그리고 `classifier`를 linear layer로 설정합니다.
그리고 `forward` 함수에서 순차적으로 사용하여 예측 결과를 반환합니다.

다음은 마지막 classifier layer를 제외한 나머지 부분을 freeze하는 코드를 구현합니다.

In [11]:
for param in model.encoder.parameters():
  param.requires_grad = False

위의 코드는 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.
즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

마지막으로 이전과 같은 코드를 사용하여 학습 결과를 확인해봅시다.

In [ ]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to('cuda')
loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 10

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 232.52257829904556
Epoch   1 | Train Loss: 198.96078765392303
Epoch   2 | Train Loss: 185.74048367142677
Epoch   3 | Train Loss: 178.2695178091526
Epoch   4 | Train Loss: 173.67855405807495
Epoch   5 | Train Loss: 170.96059101819992
Epoch   6 | Train Loss: 168.5388327538967
Epoch   7 | Train Loss: 166.85481399297714
Epoch   8 | Train Loss: 166.17386549711227
Epoch   9 | Train Loss: 164.44775652885437


In [ ]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.515 | Test acc: 0.512


Loss가 잘 떨어지고, 이전에 우리가 구현한 Transformer보다 더 빨리 수렴하는 것을 알 수 있습니다.

# 과제

Huggingface dataset의 fancyzhx/ag_news를 load합니다.

-----

In [12]:
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("fancyzhx/ag_news")

# 데이터셋 정보 출력
print(dataset)

# 첫 번째 샘플 출력
print(dataset['train'][0])

train_dataset = load_dataset("fancyzhx/ag_news", split="train")
test_dataset = load_dataset("fancyzhx/ag_news", split="test")

print(train_dataset[0])
print(test_dataset[0])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})
{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
{'text': "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.", 'label': 2}


- `collate_fn` 함수에 다음 수정사항들을 반영하면 됩니다.
    - Truncation과 관련된 부분들을 지웁니다.

In [13]:
def collate_fn(batch):
    texts, labels = [], []

    # 텍스트와 레이블 분리
    for row in batch:
        labels.append(row['label'])
        texts.append(row['text'])

    # 토큰화에서 Truncation과 max_length 제거
    texts = torch.LongTensor(tokenizer(texts, padding=True).input_ids)
    labels = torch.LongTensor(labels)

    return texts, labels

Classifier output, loss function, accuracy function 변경

뉴스 기사 분류 문제는 binary classification이 아닌 일반적인 classification 문제입니다. MNIST 과제(링크)에서 했던 것 처럼 nn.CrossEntropyLoss 를 추가하고 TextClassifier의 출력 차원을 잘 조정하여 task를 풀 수 있도록 수정하시면 됩니다.

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam

from torch.utils.data import DataLoader

# train_loader 정의
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn
)

# test_loader 정의
test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=collate_fn
)

# 1) TextClassifier 클래스 정의
class TextClassifier(nn.Module):
    def __init__(self, num_classes):
        super(TextClassifier, self).__init__()
        # 2) 사전 학습된 DistilBERT 모델 로드
        self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
        # 3) DistilBERT의 출력(768차원)을 받아 클래스 수(num_classes)만큼 출력하도록 설정
        self.classifier = nn.Linear(768, num_classes)

    def forward(self, x):
        # 4) DistilBERT 모델에 입력 x(토큰 ID 시퀀스)를 전달하여 출력 획득
        outputs = self.encoder(x)
        # 5) DistilBERT의 'last_hidden_state' (마지막 레이어의 은닉 상태) 선택
        hidden_state = outputs['last_hidden_state']
        # 6) [CLS] 토큰 위치(배치 차원[:, 0])의 벡터만 추출하여 분류기에 전달
        x = self.classifier(hidden_state[:, 0])
        return x

# 7) 정확도(accuracy) 계산 함수 정의
def accuracy_fn(preds, labels):
    # preds: 모델의 예측 로짓(logits), 크기 [batch_size, num_classes]
    # labels: 실제 정답 레이블, 크기 [batch_size]
    # 8) argmax(dim=1)을 통해 각 샘플에서 가장 큰 로짓 값의 인덱스가 예측 클래스가 됨
    correct = (preds.argmax(dim=1) == labels).sum().item()
    # 9) labels.size(0)는 배치(batch) 크기
    total = labels.size(0)
    # 10) 맞춘 개수를 전체 샘플 수로 나누어 정확도 계산
    return correct / total

# 11) 분류할 클래스 수 설정(예: 5개 클래스)
num_classes = 5

# 12) 모델 초기화(TextClassifier 인스턴스 생성) 및 GPU로 이동
model = TextClassifier(num_classes).to('cuda')

# 13) 다중 클래스 분류를 위한 CrossEntropyLoss 사용
loss_fn = nn.CrossEntropyLoss()

# 14) Adam 옵티마이저로 모델 파라미터를 학습, 학습률은 1e-3
optimizer = Adam(model.parameters(), lr=1e-3)

# 15) 학습할 epoch 수(반복 횟수) 설정
n_epochs = 10

# 16) epoch 반복 시작
for epoch in range(n_epochs):
    # 17) 학습 모드 설정(드롭아웃, 배치정규화 등 학습 모드로 동작)
    model.train()
    # 18) 현재 epoch 동안의 손실 총합을 저장할 변수
    total_loss = 0.0

    # 19) train_loader(훈련 데이터 로더)에서 배치를 하나씩 가져옴
    for data in train_loader:
        # 20) data에서 (inputs, labels)를 추출
        inputs, labels = data
        # 21) inputs, labels를 CUDA 장치(GPU)로 이동 및 레이블을 long 타입으로 변환
        inputs, labels = inputs.to('cuda'), labels.to('cuda').long()

        # 22) 모델에 입력을 전달하여 예측 로짓 preds를 얻음
        preds = model(inputs)
        # 23) CrossEntropyLoss를 이용해 손실(loss) 계산
        loss = loss_fn(preds, labels)

        # 24) 역전파(Backprop) 전, 기존에 계산된 그래디언트(gradient) 초기화
        optimizer.zero_grad()
        # 25) 역전파를 통해 손실(loss)에 대한 모델 파라미터의 그래디언트 계산
        loss.backward()
        # 26) Adam 옵티마이저를 통해 모델 파라미터 업데이트
        optimizer.step()

        # 27) 배치 손실을 total_loss에 누적
        total_loss += loss.item()

    # 28) epoch 별 평균 손실(Train Loss)을 계산
    avg_loss = total_loss / len(train_loader)
    # 29) 현재 epoch의 학습 손실을 출력
    print(f"Epoch {epoch+1:2d}/{n_epochs} | Train Loss: {avg_loss:.4f}")

# 30) 학습을 마친 모델로 테스트 정확도를 측정하기 위해 평가 모드 설정
model.eval()
# 31) 테스트 정확도 합산 변수
total_acc = 0.0

# 32) 테스트 시에는 기울기 계산을 하지 않으므로 no_grad() 사용
with torch.no_grad():
    # 33) test_loader(테스트 데이터 로더)에서 배치를 하나씩 가져옴
    for data in test_loader:
        # 34) (inputs, labels) 추출 후 GPU로 이동
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda').long()

        # 35) 모델에 입력하여 예측 로짓 preds 얻음
        preds = model(inputs)
        # 36) accuracy_fn(정확도 함수)로 정확도를 계산
        total_acc += accuracy_fn(preds, labels)

# 37) 전체 배치에 대한 평균 정확도 계산
avg_test_acc = total_acc / len(test_loader)
# 38) 최종 테스트 정확도 출력
print(f"\nFinal Test Accuracy: {avg_test_acc:.4f}")


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch  1/10 | Train Loss: 1.3968
Epoch  2/10 | Train Loss: 1.3882
Epoch  3/10 | Train Loss: 1.3895


In [ ]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import DistilBertTokenizer

# 데이터셋 로드 (fancyzhx/ag_news 예제)
dataset = load_dataset("fancyzhx/ag_news")

# DistilBERT 토크나이저 로드
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# collate_fn 함수 정의
def collate_fn(batch):
    texts, labels = [], []

    # 텍스트와 레이블 분리
    for row in batch:
        labels.append(row['label'])
        texts.append(row['text'])

    # 토큰화에서 Truncation과 max_length 제거
    texts = torch.LongTensor(tokenizer(texts, padding=True).input_ids)
    labels = torch.LongTensor(labels)

    return texts, labels

# DataLoader 설정
train_loader = DataLoader(
    dataset["train"],  # 훈련 데이터셋
    batch_size=16,     # 배치 크기
    shuffle=True,      # 데이터 셔플링
    collate_fn=collate_fn  # 사용자 정의 collate_fn
)

# 테스트 DataLoader
test_loader = DataLoader(
    dataset["test"],  # 테스트 데이터셋
    batch_size=16,    # 배치 크기
    shuffle=False,    # 셔플링 비활성화
    collate_fn=collate_fn  # 사용자 정의 collate_fn
)

for texts, labels in train_loader:
    print(f"Texts shape: {texts.shape}")  # 텍스트 텐서 크기
    print(f"Labels shape: {labels.shape}")  # 레이블 텐서 크기
    print(f"First text tensor: {texts[0]}")  # 첫 번째 텍스트 토큰 ID
    print(f"First label: {labels[0]}")  # 첫 번째 레이블
    break

Texts shape: torch.Size([16, 60])
Labels shape: torch.Size([16])
First text tensor: tensor([  101,  2007,  7064,  8455,  1010,  3516,  4455,  2005, 13982,  2015,
         2055,  3998,  1010,  2199,  3901,  1999,  5780,  2752,  1997,  5340,
         3509,  2221,  2020,  2409,  2000, 22811,  3225,  1016,  1052,  1012,
         1049,  1012,  9432,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
First label: 0
